In [1]:
import numpy as np

state = 3

theta = np.array([i/state*2*np.pi for i in range(state)])

moment = np.array([np.exp(angle*1j) for angle in theta])

In [2]:
print(moment)
print(moment**2)

[ 1. +0.j        -0.5+0.8660254j -0.5-0.8660254j]
[ 1. +0.j        -0.5-0.8660254j -0.5+0.8660254j]


In [3]:
a = np.array([1.0, 2.0])
b = np.array([1.0+1.0j, 2.0+1.0j])

print(np.einsum("i,j->ij",a,b))

[[1.+1.j 2.+1.j]
 [2.+2.j 4.+2.j]]


In [4]:
import numpy as np
import numpy.typing as npt

def efficient_einsum(
    T1: npt.NDArray[np.float64],
    T2: npt.NDArray[np.float64 | np.complex128],
    T3: npt.NDArray[np.float64],
    T4: npt.NDArray[np.float64 | np.complex128],
    mode: str,
) -> npt.NDArray[np.complex128]:

    if mode == "UD":
        left = np.einsum("iae,ejgb->ijgba", T1, T2, optimize=True)
        right = np.einsum("cgk,abcl->abgkl", T3, T4, optimize=True)

        return np.einsum("ijgba,abgkl->ijkl", left, right, optimize=True)

    elif mode == "LR":
        up = np.einsum("ldg,ibcd->libcg", T1, T2, optimize=True)
        down = np.einsum("bej,cekg->bcgkj", T3, T4, optimize=True)

        return np.einsum("libcg,bcgkj->ijkl", up, down, optimize=True)

In [5]:
size = 8

trunc_U, trunc_U_T = np.random.rand(size,size,size), np.random.rand(size,size,size)
pure, first_order = np.random.rand(size,size,size,size), np.random.rand(size,size,size,size)

In [6]:
%timeit np.einsum("ldg,ibcd,bej,cekg->ijkl", trunc_U_T, first_order, trunc_U, pure, optimize=True)

In [ ]:
%timeit efficient_einsum(trunc_U_T, first_order, trunc_U, pure, mode="LR")

565 µs ± 68.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
print(np.allclose(
    np.einsum("ldg,ibcd,bej,cekg->ijkl", trunc_U_T, first_order, trunc_U, pure, optimize=True),
    efficient_einsum(trunc_U_T, first_order, trunc_U, pure, mode="LR")
))

True


In [ ]:
%timeit np.einsum("iae,ejgb,cgk,abcl->ijkl", trunc_U_T, first_order, trunc_U, pure, optimize=True)

870 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit efficient_einsum(trunc_U_T, first_order, trunc_U, pure, mode="UD")

605 µs ± 70.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
print(np.allclose(
    np.einsum("iae,ejgb,cgk,abcl->ijkl", trunc_U_T, first_order, trunc_U, pure, optimize=True),
    efficient_einsum(trunc_U_T, first_order, trunc_U, pure, mode="UD")
))

True


In [ ]:
print(np.allclose(
    np.einsum("iae,ejgb,cgk,abcl->ijkl", trunc_U_T, pure, trunc_U, first_order, optimize=True),
    efficient_einsum(trunc_U_T, pure, trunc_U, first_order, mode="UD")
))

True


In [1]:
import numpy as np

from src.initial_state import get_pure_state
from src.dataclass import Model, HOTRG, Input, Mid_Time
from src.HOSVD import HOSVD
from tensorly.decomposition import tucker

mid_time = Mid_Time(create=[], reshape=[], matmul=[],
                    decompose=[], truncate=[], total=[])

In [2]:
model = Model(state=2, beta=1.0, magnetic_field=0.0)

pure = get_pure_state(model)
print(pure.shape)

(2, 2, 2, 2)


In [3]:
new_pure = np.einsum("abcd,efgb->aefgcd", pure, pure, optimize=True)
print(new_pure.shape)

(2, 2, 2, 2, 2, 2)


In [4]:
U, epsilon = HOSVD(new_pure, [3,4], [0,5,1,2], bond=2, mid_time=mid_time)

[[1976.10007332    0.            0.         1436.86269558]
 [   0.         1489.47916125 1435.8989914     0.        ]
 [   0.         1435.8989914  1489.47916125    0.        ]
 [1436.86269558    0.            0.         1124.09118053]]
True


In [5]:
print(U)
print(epsilon)
print(np.allclose(np.diag([1.0 for _ in range(4)]), U@U.T))
print(np.allclose(np.diag([1.0 for _ in range(4)]), U.T@U))

[[-0.80132775  0.         -0.         -0.59822558]
 [ 0.         -0.70710678  0.70710678  0.        ]
 [ 0.         -0.70710678 -0.70710678  0.        ]
 [-0.59822558  0.          0.          0.80132775]]
104.99164249938042
True
True


In [1]:
import numpy as np
import numpy.typing as npt
from typing import Any
import matplotlib.pyplot as plt
import time
import sys

from abs_path import abs_path
sys.path.append(f"{abs_path.abs_path}")
del sys.path[0]
del sys.path[3]
print(sys.path)

['/home/sungbinlee/miniconda3/envs/TRG/lib/python311.zip', '/home/sungbinlee/miniconda3/envs/TRG/lib/python3.11', '/home/sungbinlee/miniconda3/envs/TRG/lib/python3.11/lib-dynload', '/home/sungbinlee/miniconda3/envs/TRG/lib/python3.11/site-packages', '/mnt/d/Research/TN']


In [2]:
from HOTRG.src.multiprocessing import multiprocessing
from lib.base_function import exact_free_energy

In [3]:
Datapoint = 50
state = 2
Kc = np.log(1+np.sqrt(state))*(state-1)/state
coupling_max = np.round(2*Kc,3)
beta = np.linspace(
    start=coupling_max/Datapoint, stop=coupling_max, num=Datapoint)[::-1]
magnetic_field = np.full(Datapoint, 0.0)[::-1]
print(beta)

[0.881   0.86338 0.84576 0.82814 0.81052 0.7929  0.77528 0.75766 0.74004
 0.72242 0.7048  0.68718 0.66956 0.65194 0.63432 0.6167  0.59908 0.58146
 0.56384 0.54622 0.5286  0.51098 0.49336 0.47574 0.45812 0.4405  0.42288
 0.40526 0.38764 0.37002 0.3524  0.33478 0.31716 0.29954 0.28192 0.2643
 0.24668 0.22906 0.21144 0.19382 0.1762  0.15858 0.14096 0.12334 0.10572
 0.0881  0.07048 0.05286 0.03524 0.01762]


In [ ]:
step, Dcut = 20, 12

free_energy = multiprocessing(state=state, beta=beta, magnetic_field=magnetic_field, 
              step=step, Dcut=Dcut, max_workers=2)